In [1]:
import requests
import json
import pymongo
from key import key
url = "https://api-basketball.p.rapidapi.com/"
headers = {
    "X-RapidAPI-Host": "api-basketball.p.rapidapi.com",
    "X-RapidAPI-Key": key
}

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.nba_db

In [4]:
seasons = ['2019-2020', '2020-2021', '2021-2022']
collection = db.standings
collection.delete_many({})
for season in seasons:
    params = f'standings?league=12&season={season}'
    response = requests.get(url + params, headers=headers).json()

    weststandings = {}
    eaststandings = {}

    for team in response["response"][0]:

        if team["group"]["name"] == "Western Conference":
            weststandings[team["team"]["name"]] = {"wins": team["games"]["win"]["total"],
                                  "losses": team["games"]["lose"]["total"], "logo": team["team"]["logo"]}
        if team["group"]["name"] == "Eastern Conference":
            eaststandings[team["team"]["name"]] = {"wins": team["games"]["win"]["total"],
                                  "losses": team["games"]["lose"]["total"], "logo" : team["team"]["logo"]}
    collection.insert_one({"season" : season, "data" : {"weststandings" : weststandings, "eaststandings": eaststandings}})

In [5]:
idmap = {}
for team in response["response"][0]:
    idmap[team["team"]["id"]] = team["team"]["name"]

In [6]:
idmap

{155: 'Phoenix Suns',
 146: 'Memphis Grizzlies',
 141: 'Golden State Warriors',
 138: 'Dallas Mavericks',
 160: 'Utah Jazz',
 139: 'Denver Nuggets',
 149: 'Minnesota Timberwolves',
 144: 'Los Angeles Clippers',
 150: 'New Orleans Pelicans',
 158: 'San Antonio Spurs',
 145: 'Los Angeles Lakers',
 157: 'Sacramento Kings',
 156: 'Portland Trail Blazers',
 152: 'Oklahoma City Thunder',
 142: 'Houston Rockets',
 147: 'Miami Heat',
 148: 'Milwaukee Bucks',
 133: 'Boston Celtics',
 154: 'Philadelphia 76ers',
 159: 'Toronto Raptors',
 136: 'Chicago Bulls',
 137: 'Cleveland Cavaliers',
 134: 'Brooklyn Nets',
 132: 'Atlanta Hawks',
 135: 'Charlotte Hornets',
 151: 'New York Knicks',
 161: 'Washington Wizards',
 143: 'Indiana Pacers',
 140: 'Detroit Pistons',
 153: 'Orlando Magic'}

In [7]:
collection = db.h2h
collection.delete_many({})
for team1 in range(132, 162):
    for team2 in range(team1 + 1, 162):
        games = []
        params = f"/games?season=2020-2021&h2h={team1}-{team2}"
        response = requests.get(url + params, headers=headers).json()["response"]
        for game in response:
            simplegame = {"date" : game["date"][:game["date"].index('T')], 
                          "home" : {"team": game["teams"]["home"]["name"], "score": game["scores"]["home"]["total"]},
                          "away" : {"team": game["teams"]["away"]["name"], "score": game["scores"]["away"]["total"]}}
            games.append(simplegame)
        collection.insert_one({'teams': [idmap[team1], idmap[team2]], "data": games})

In [11]:
collection = db.teamlist
collection.delete_many({})
teams = []
for team in range(132,162):
    teams.append(idmap[team])
    
collection.insert_one({'teamlist' : teams})

In [12]:
with open('player_stats.json') as file:
    file_data = json.load(file)

In [13]:
collection = db.player_data
collection.insert_many(file_data)